# Ontology Extension for nbdev

> An extension to nbdev for ontology and vocabulary development using notebooks.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## Basic Imports

In [ ]:
#| export
from nbdev.process import *
from nbdev.maker import *
from execnb.nbio import read_nb, NbCell  # Updated import
from rdflib import Graph, URIRef, Literal, Namespace
from pathlib import Path
from fastcore.test import *
from fastcore.script import call_parse  # Add this import
import json
import re
from markdown_it import MarkdownIt
from markdown_it.tree import SyntaxTreeNode


## OntologyGraph

In [ ]:
#| export
class OntologyGraph:
    "Builds and manages the RDF graph for ontology content"
    def __init__(self):
        self.graph = Graph()
        
    def add_cell_content(self, cell):
        "Add content from a cell to the graph"
        if cell.cell_type == 'markdown' and 'export_mdld' in cell.directives_:
            self._add_markdown_ld(cell.source)
        elif cell.cell_type == 'code':
            if 'export_ttl' in cell.directives_:
                self._add_turtle(cell.source)
            elif 'export_jsonld' in cell.directives_:
                self._add_jsonld(cell.source)
    
    def _add_markdown_ld(self, text):
        # Use our local function to parse and add
        triples, prefixes, base_uri = markdown_ld_to_triples(text)
        # Add prefixes to graph
        for prefix, uri in prefixes.items():
            self.graph.bind(prefix, Namespace(uri))
        # Add base URI if provided
        if base_uri:
            self.graph.base = base_uri
        # Add triples to graph
        for s, p, o in triples:
            # Handle URIs vs literals
            if s.startswith('<') and s.endswith('>'): s = s[1:-1]
            if p.startswith('<') and p.endswith('>'): p = p[1:-1]
            
            if o.startswith('"'):
                # It's a literal
                obj_val = Literal(o.strip('"'))
            elif o.isdigit():
                obj_val = Literal(int(o))
            else:
                if o.startswith('<') and o.endswith('>'): o = o[1:-1]
                obj_val = URIRef(o)
                
            self.graph.add((URIRef(s), URIRef(p), obj_val))

            
    def _add_turtle(self, text):
        # Parse Turtle directly into the graph
        self.graph.parse(data=text, format="turtle")
    
    def _add_jsonld(self, text):
        # Parse JSON-LD directly into the graph
        self.graph.parse(data=text, format="json-ld")
    
    def serialize(self, format="turtle"):
        "Serialize the graph to the specified format"
        if format == "jsonld-context":
            # Generate a JSON-LD context from the graph
            # This would need custom implementation
            return self._generate_jsonld_context()
        else:
            return self.graph.serialize(format=format)

In [ ]:
# Test basic functionality of OntologyGraph
test_graph = OntologyGraph()
test_turtle = """
@prefix ex: <http://example.org/> .
ex:Subject ex:predicate "object" .
"""
test_graph._add_turtle(test_turtle)
test_eq(len(test_graph.graph), 1)
serialized = test_graph.serialize(format="turtle")
test_eq("ex:Subject" in serialized, True)


## Ontology Maker
OntologyMaker class that will handle exporting the graph to files

In [ ]:
#| export
class OntologyMaker:
    "Export ontology graph to various formats"
    def __init__(self, dest, name, format="turtle"):
        self.dest = Path(dest)
        self.name = name
        self.format = format
        self.dest.mkdir(parents=True, exist_ok=True)
        
    def make(self, graph):
        "Export the graph to the specified format"
        # Determine file extension based on format
        ext = self._get_extension()
        filename = f"{self.name}.{ext}"
        filepath = self.dest / filename
        
        # Serialize and write to file
        serialized = graph.serialize(format=self.format)
        with open(filepath, 'w') as f:
            f.write(serialized)
        
        return filepath
    
    def _get_extension(self):
        "Get file extension based on format"
        format_map = {
            "turtle": "ttl",
            "xml": "rdf",
            "json-ld": "jsonld",
            "n3": "n3",
            "nt": "nt",
            "nquads": "nq"
        }
        return format_map.get(self.format, self.format)


In [ ]:
#| export
def extract_prefixes(ast):
    """Extract prefixes and base URI from a Markdown-ld AST."""
    prefixes = {}
    base_uri = None
    
    # Look for level-1 heading followed by a code block
    for i, node in enumerate(ast.children):
        if node.type == 'heading' and node.tag == 'h1':
            # Check if next node is a code block
            if i+1 < len(ast.children) and ast.children[i+1].type == 'code':
                code_block = ast.children[i+1]
                # Parse the code block content for prefixes
                for line in code_block.content.split('\n'):
                    line = line.strip()
                    if line.startswith('@prefix'):
                        # Extract prefix and URI
                        parts = line.split(' ', 2)
                        if len(parts) >= 3:
                            prefix = parts[1].rstrip(':')
                            uri = parts[2].rstrip(' .')
                            if uri.startswith('<') and uri.endswith('>'): 
                                uri = uri[1:-1]
                            prefixes[prefix] = uri
                    elif line.startswith('@base'):
                        # Extract base URI
                        parts = line.split(' ', 1)
                        if len(parts) >= 2:
                            uri = parts[1].rstrip(' .')
                            if uri.startswith('<') and uri.endswith('>'): 
                                uri = uri[1:-1]
                            base_uri = uri
    
    return prefixes, base_uri

In [ ]:
#| export
def extract_triples(ast):
    """Extract RDF triples from a Markdown-ld AST."""
    triples = []
    current_subject = None
    current_predicate = None
    
    # Helper function to extract text from inline code
    def extract_code_value(node):
        for child in node.children:
            if child.type == 'inline':
                for inline_child in child.children:
                    if inline_child.type == 'code_inline':
                        return inline_child.content
        return None
    
    # Process each node in the AST
    for node in ast.children:
        # Handle level-2 headings (subjects)
        if node.type == 'heading' and node.tag == 'h2':
            # The next node should be a paragraph with inline code
            if node.next_sibling and node.next_sibling.type == 'paragraph':
                current_subject = extract_code_value(node.next_sibling)
                current_predicate = None
        
        # Handle level-3 headings (predicates)
        elif node.type == 'heading' and node.tag == 'h3':
            if node.next_sibling and node.next_sibling.type == 'paragraph':
                current_predicate = extract_code_value(node.next_sibling)
        
        # Handle bullet lists (objects)
        elif node.type == 'bullet_list' and current_subject and current_predicate:
            for list_item in node.children:
                if list_item.type == 'list_item':
                    # Extract object from list item
                    obj = None
                    for paragraph in list_item.children:
                        if paragraph.type == 'paragraph':
                            obj = extract_code_value(paragraph)
                    
                    if obj:
                        triples.append((current_subject, current_predicate, obj))
    
    return triples

In [ ]:
#| export
def markdown_ld_to_triples(markdown_text):
    """Convert Markdown-LD text to RDF triples."""
    from markdown_it import MarkdownIt
    from markdown_it.tree import SyntaxTreeNode
    
    md = MarkdownIt()
    tokens = md.parse(markdown_text)
    ast = SyntaxTreeNode(tokens)
    triples = extract_triples(ast)
    prefixes, base_uri = extract_prefixes(ast)
    return triples, prefixes, base_uri


In [ ]:
# Test with Markdown-LD
test_mdld = """
# Test Ontology

```
@prefix ex: <http://example.org/> .
@base <http://example.org/base/> .
```

## TestClass

`ex:TestClass`

```
ex:TestClass a owl:Class .
```

### has property

`ex:hasProperty`

- `"test value"`
"""

In [ ]:
test_graph = OntologyGraph()
test_graph._add_markdown_ld(test_mdld)
test_eq(len(test_graph.graph) > 0, True)

In [ ]:
#| export
class OntologyProcessor(Processor):
    "Process cells with ontology directives"
    def __init__(self, nb):
        super().__init__(nb)
        self.graph = OntologyGraph()
        
    def _export_owl_(self, cell):
        "Process cells marked with #| export_owl"
        self.graph.add_cell_content(cell)
        
    def _export_ttl_(self, cell):
        "Process cells marked with #| export_ttl"
        self.graph.add_cell_content(cell)
        
    def _export_mdld_(self, cell):
        "Process cells marked with #| export_mdld"
        self.graph.add_cell_content(cell)
        
    def get_graph(self):
        "Return the constructed RDF graph"
        return self.graph


In [ ]:
# Test OntologyProcessor with a notebook cell
import nbformat
from execnb.nbio import NbCell

# Create a test cell using nbformat first
nb_cell = nbformat.v4.new_code_cell(source="""
@prefix ex: <http://example.org/> .
ex:TestSubject ex:testPredicate "test object" .
""")

# Now create a NbCell with the required arguments
test_cell = NbCell(0, nb_cell)  # idx=0, cell=nb_cell
test_cell.directives_ = {'export_ttl': []}

# Create a processor and process the cell
processor = OntologyProcessor(None)  # nb parameter not used in this test
processor._export_ttl_(test_cell)

# Check if the triple was added to the graph
test_eq(len(processor.graph.graph), 1)


In [ ]:
# Test OntologyMaker
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    # Create a graph with a test triple
    test_graph = OntologyGraph()
    test_graph._add_turtle("""
    @prefix ex: <http://example.org/> .
    ex:TestSubject ex:testPredicate "test object" .
    """)
    
    # Export the graph
    maker = OntologyMaker(tmpdir, "test_ontology")
    filepath = maker.make(test_graph.graph)  # Note: We pass the RDFLib graph, not the OntologyGraph
    
    # Check that the file was created and contains our triple
    test_eq(Path(filepath).exists(), True)
    with open(filepath, 'r') as f:
        content = f.read()
    test_eq("TestSubject" in content, True)


In [ ]:
#| export
def process_ontology_notebook(nb_path, output_dir=None, formats=None):
    """Process a notebook and export ontology content to files.
    
    Args:
        nb_path: Path to the notebook
        output_dir: Directory to output the ontology files (defaults to same dir as notebook)
        formats: List of output formats (defaults to ["turtle"])
        
    Returns:
        Dictionary mapping formats to output file paths
    """
    # Set defaults
    if formats is None: formats = ["turtle"]
    nb_path = Path(nb_path)
    if output_dir is None: output_dir = nb_path.parent
    
    # Read the notebook
    nb = read_nb(nb_path)
    
    # Process the notebook
    processor = OntologyProcessor(nb)
    for cell in nb.cells:
        if not hasattr(cell, 'directives_'):
            cell.directives_ = extract_directives(cell)
        
        # Use 'in' instead of 'get' to check for directive presence
        if 'export_ttl' in cell.directives_:
            processor._export_ttl_(cell)
        elif 'export_owl' in cell.directives_:
            processor._export_owl_(cell)
        elif 'export_mdld' in cell.directives_:
            processor._export_mdld_(cell)
    
    # Get the graph
    graph = processor.graph.graph
    
    # Export to each format
    results = {}
    for fmt in formats:
        maker = OntologyMaker(output_dir, nb_path.stem, format=fmt)
        filepath = maker.make(graph)
        results[fmt] = filepath
    
    return results


In [ ]:
# Test process_ontology_notebook with the correct string to check
import nbformat
import tempfile
from pathlib import Path

with tempfile.TemporaryDirectory() as tmpdir:
    # Create a test notebook
    nb = nbformat.v4.new_notebook()
    
    # Add a cell with Turtle content
    cell = nbformat.v4.new_code_cell(source="""#| export_ttl
@prefix ex: <http://example.org/> .
ex:TestSubject ex:testPredicate "test object" .
""")
    nb.cells.append(cell)
    
    # Write the notebook to a file
    nb_path = Path(tmpdir) / "test_ontology.ipynb"
    with open(nb_path, 'w') as f:
        nbformat.write(nb, f)
    
    # Process the notebook
    results = process_ontology_notebook(nb_path)
    
    # Check that the output file was created
    test_eq(len(results), 1)
    test_eq("turtle" in results, True)
    test_eq(Path(results["turtle"]).exists(), True)
    
    # Check the content
    with open(results["turtle"], 'r') as f:
        content = f.read()
    
    # Print the content for debugging
    print("File content:")
    print(content)
    
    # Check for the correct string pattern
    test_eq("ex:TestSubject" in content, True)



File content:
@prefix ex: <http://example.org/> .

ex:TestSubject ex:testPredicate "test object" .




In [ ]:
#| export
@call_parse
def nbvoc_export(
    path:str=None, # Path to notebook or directory containing notebooks (default: current dir)
    recursive:bool=True, # Search recursively in directories
    formats:str='turtle', # Output formats (comma-separated list)
    output_dir:str=None, # Output directory (default: ontology_path from settings.ini)
):
    "Export ontology content from notebooks"
    from nbdev.config import get_config
    
    path = Path(path or '.')
    format_list = formats.split(',')
    
    # Use ontology_path from settings.ini or default to 'ontology'
    if output_dir is None:
        try:
            output_dir = get_config().get('ontology_path', 'ontology')
        except:
            output_dir = 'ontology'
    
    # Create output directory if it doesn't exist
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Process a single notebook
    if path.is_file() and path.suffix == '.ipynb':
        # Preserve directory structure relative to current directory
        rel_path = path.relative_to(Path.cwd())
        # Get the parent directory for the output
        rel_dir = rel_path.parent
        # Create parent directory if needed
        notebook_output_dir = output_path / rel_dir
        notebook_output_dir.mkdir(parents=True, exist_ok=True)
        
        results = process_ontology_notebook(path, notebook_output_dir, format_list)
        print(f"Exported {path.name} to: {', '.join(str(p) for p in results.values())}")
        return
    
    # Process a directory of notebooks
    if path.is_dir():
        pattern = '**/*.ipynb' if recursive else '*.ipynb'
        notebooks = list(path.glob(pattern))
        if not notebooks:
            print(f"No notebooks found in {path}")
            return
            
        for nb_path in notebooks:
            try:
                # Preserve directory structure relative to search path
                rel_path = nb_path.relative_to(path)
                # Get the parent directory for the output
                rel_dir = rel_path.parent
                # Create parent directory if needed
                notebook_output_dir = output_path / rel_dir
                notebook_output_dir.mkdir(parents=True, exist_ok=True)
                
                results = process_ontology_notebook(nb_path, notebook_output_dir, format_list)
                print(f"Exported {nb_path.name} to: {', '.join(str(p) for p in results.values())}")
            except Exception as e:
                print(f"Error processing {nb_path}: {e}")
        
        return
    
    print(f"Path not found or not a notebook: {path}")


In [ ]:
# Test the CLI function with directory structure preservation
import tempfile
import nbformat
import os
from pathlib import Path
import shutil

with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)
    
    # Create a directory structure
    notebooks_dir = tmp_path / "notebooks"
    notebooks_dir.mkdir()
    
    subdir1 = notebooks_dir / "module1"
    subdir1.mkdir()
    
    subdir2 = notebooks_dir / "module2" / "submodule"
    subdir2.mkdir(parents=True)
    
    # Create test notebooks in different directories
    nb1 = nbformat.v4.new_notebook()
    cell1 = nbformat.v4.new_code_cell(source="""#| export_ttl
@prefix ex: <http://example.org/> .
ex:Module1 ex:hasProperty "value1" .
""")
    nb1.cells.append(cell1)
    
    nb2 = nbformat.v4.new_notebook()
    cell2 = nbformat.v4.new_code_cell(source="""#| export_ttl
@prefix ex: <http://example.org/> .
ex:Module2 ex:hasProperty "value2" .
""")
    nb2.cells.append(cell2)
    
    # Write notebooks to their respective directories
    nb_path1 = subdir1 / "ontology1.ipynb"
    with open(nb_path1, 'w') as f:
        nbformat.write(nb1, f)
    
    nb_path2 = subdir2 / "ontology2.ipynb"
    with open(nb_path2, 'w') as f:
        nbformat.write(nb2, f)
    
    # Create output directory
    ontology_dir = tmp_path / "ontology"
    
    # Run the export function
    print("\nTesting export with directory structure preservation:")
    current_dir = os.getcwd()
    try:
        # Change to the temp directory to test relative paths
        os.chdir(tmp_path)
        nbvoc_export("notebooks", output_dir="ontology", formats="turtle")
    finally:
        # Change back to original directory
        os.chdir(current_dir)
    
    # Check output directory structure
    expected_files = [
        ontology_dir / "module1" / "ontology1.ttl",
        ontology_dir / "module2" / "submodule" / "ontology2.ttl"
    ]
    
    for file_path in expected_files:
        print(f"Checking if {file_path} exists...")
        test_eq(file_path.exists(), True)
        
        # Verify content
        content = file_path.read_text()
        if "Module1" in content:
            test_eq("ex:Module1" in content, True)
        elif "Module2" in content:
            test_eq("ex:Module2" in content, True)
    
    print("Directory structure test passed!")



Testing export with directory structure preservation:
Exported ontology1.ipynb to: ontology/module1/ontology1.ttl
Exported ontology2.ipynb to: ontology/module2/submodule/ontology2.ttl
Checking if /var/folders/qp/nh72_hpd4876lr8t6k44015c0000gn/T/tmpvcvvctss/ontology/module1/ontology1.ttl exists...
Checking if /var/folders/qp/nh72_hpd4876lr8t6k44015c0000gn/T/tmpvcvvctss/ontology/module2/submodule/ontology2.ttl exists...
Directory structure test passed!


In [ ]:
# Test the CLI function with directory structure preservation
import tempfile
import nbformat
import os
from pathlib import Path
import shutil

with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)
    
    # Create a directory structure
    notebooks_dir = tmp_path / "notebooks"
    notebooks_dir.mkdir()
    
    subdir1 = notebooks_dir / "module1"
    subdir1.mkdir()
    
    subdir2 = notebooks_dir / "module2" / "submodule"
    subdir2.mkdir(parents=True)
    
    # Create test notebooks in different directories
    nb1 = nbformat.v4.new_notebook()
    cell1 = nbformat.v4.new_code_cell(source="""#| export_ttl
@prefix ex: <http://example.org/> .
ex:Module1 ex:hasProperty "value1" .
""")
    nb1.cells.append(cell1)
    
    nb2 = nbformat.v4.new_notebook()
    cell2 = nbformat.v4.new_code_cell(source="""#| export_ttl
@prefix ex: <http://example.org/> .
ex:Module2 ex:hasProperty "value2" .
""")
    nb2.cells.append(cell2)
    
    # Write notebooks to their respective directories
    nb_path1 = subdir1 / "ontology1.ipynb"
    with open(nb_path1, 'w') as f:
        nbformat.write(nb1, f)
    
    nb_path2 = subdir2 / "ontology2.ipynb"
    with open(nb_path2, 'w') as f:
        nbformat.write(nb2, f)
    
    # Create output directory
    ontology_dir = tmp_path / "ontology"
    
    # Run the export function
    print("\nTesting export with directory structure preservation:")
    current_dir = os.getcwd()
    try:
        # Change to the temp directory to test relative paths
        os.chdir(tmp_path)
        nbvoc_export("notebooks", output_dir="ontology", formats="turtle")
    finally:
        # Change back to original directory
        os.chdir(current_dir)
    
    # Check output directory structure - updated paths
    expected_files = [
        ontology_dir / "module1" / "ontology1.ttl",
        ontology_dir / "module2" / "submodule" / "ontology2.ttl"
    ]
    
    for file_path in expected_files:
        print(f"Checking if {file_path} exists...")
        test_eq(file_path.exists(), True)
        
        # Verify content
        content = file_path.read_text()
        if "Module1" in str(file_path):
            test_eq("ex:Module1" in content, True)
        elif "Module2" in str(file_path):
            test_eq("ex:Module2" in content, True)
    
    print("Directory structure test passed!")



Testing export with directory structure preservation:
Exported ontology1.ipynb to: ontology/module1/ontology1.ttl
Exported ontology2.ipynb to: ontology/module2/submodule/ontology2.ttl
Checking if /var/folders/qp/nh72_hpd4876lr8t6k44015c0000gn/T/tmpeowe_1xp/ontology/module1/ontology1.ttl exists...
Checking if /var/folders/qp/nh72_hpd4876lr8t6k44015c0000gn/T/tmpeowe_1xp/ontology/module2/submodule/ontology2.ttl exists...
Directory structure test passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()